In [88]:
import numpy as np
import pandas as pd

# Recreate the optode spatial array as a list of lists
optode_array = np.array([
    ['', 'D05', 'S01', 'D17', '', '', ''],
    ['', '', 'D03', 'D02', '', '', ''],
    ['', 'S04', 'D04', 'S03', 'D01', 'S02', ''],
    ['', 'D06', '', '', 'D18', '', ''],
    ['S09', 'D05', 'D07', 'O01', 'D19', 'S17', 'S21'],
    ['D13', 'D09', 'D08', '', 'D20', 'D21', 'D25'],
    ['S10', 'S06', '', '', 'S29', 'S18', 'S22'],
    ['D14', 'D10', 'S25', '', 'S30', 'D22', 'D26'],
    ['S11', 'S07', 'S26', 'O02', 'S30', 'S19', 'S23'],
    ['D15', 'D11', '', '', '', 'D23', 'D27'],
    ['S12', 'S08', 'S27', '', 'S32', 'S20', 'S24'],
    ['D16', 'D12', '', '', '', 'S24', 'S28'],
    ['D29', 'S13', 'S14', 'O03', '','S15', 'S16'],
    ['', '', '', 'S28', '', '', '']
])

#make the optode array larger but retain the distance between optodes
# Get original dimensions
rows, cols = optode_array.shape

# Create new expanded array
expanded_array = np.full((2 * rows, 2 * cols), '', dtype=object)

# Populate the expanded array
for i in range(rows):
    for j in range(cols):
        expanded_array[2 * i, 2 * j] = optode_array[i, j]

print(expanded_array)




[[np.str_('') '' np.str_('D05') '' np.str_('S01') '' np.str_('D17') ''
  np.str_('') '' np.str_('') '' np.str_('') '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '']
 [np.str_('') '' np.str_('') '' np.str_('D03') '' np.str_('D02') ''
  np.str_('') '' np.str_('') '' np.str_('') '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '']
 [np.str_('') '' np.str_('S04') '' np.str_('D04') '' np.str_('S03') ''
  np.str_('D01') '' np.str_('S02') '' np.str_('') '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '']
 [np.str_('') '' np.str_('D06') '' np.str_('') '' np.str_('') ''
  np.str_('D18') '' np.str_('') '' np.str_('') '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '']
 [np.str_('S09') '' np.str_('D05') '' np.str_('D07') '' np.str_('O01') ''
  np.str_('D19') '' np.str_('S17') '' np.str_('S21') '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '']
 [np.str_('D13') '' np.str_('D09') '' np.str_('D08') '' np.str_('') ''
  np.str_('D20') '' np.str_('D21') '' np.str_('D25') '']
 ['' '' '' '' '' '' '' '' '' '' '' '' ''

In [89]:
#load the channel info
channel_info = pd.read_csv('data/channel_info.csv')
channel_info.head(2)


,source,detector,type,channel
0,1,2,'hbo',Ch1
1,1,2,'hbr',Ch2


In [ ]:
#approximate the coordinates of the channel
coords_dict = []
for row in channel_info.iterrows():
    channel = row[1]['channel']
    source = row[1]['source']
    detector = row[1]['detector']
    
    #format to match the optode array
    source = "S" + str(source).zfill(2)
    detector = "D" + str(detector).zfill(2)
    
    #find the coordinates of the source and detector
    source_coords = np.where(optode_array == source)
    detector_coords = np.where(optode_array == detector)
    if len(source_coords[0]) == 0 or len(detector_coords[0]) == 0:
        continue
    

    #channel loaction in the middle of the source and detector
    channel_x = (source_coords[0][0] + detector_coords[0][0]) / 2
    channel_y = (source_coords[1][0] + detector_coords[1][0]) / 2
    
    coords_dict.append((int(channel[2:]), int(channel_x), int(channel_y)))
    






In [98]:
#divide the coords into 4 quadrants
coords_dict = np.array(coords_dict)


quadrant_1 = coords_dict[(coords_dict[:,1] < 7) & (coords_dict[:,2] < 4)]
quadrant_2 = coords_dict[(coords_dict[:,1] >= 7) & (coords_dict[:,2] < 4)]
quadrant_3 = coords_dict[(coords_dict[:,1] < 7) & (coords_dict[:,2] >= 4)]
quadrant_4 = coords_dict[(coords_dict[:,1] >= 7) & (coords_dict[:,2] >= 4)]
print(quadrant_1.size, quadrant_2.size, quadrant_3.size, quadrant_4.size)



138 102 126 60


In [92]:
import numpy as np

def find_nearest_empty(grid, x, y, max_dist=5):
    """
    Find the nearest empty spot to (x, y) within a given max_dist range.
    Uses a breadth-first search approach.
    """
    rows, cols = grid.shape
    
    queue = [(x, y, 0)]  # (row, col, distance)

    visited = set()
    visited.add((x, y))

    while queue:
        r, c, dist = queue.pop(0)
        
        if grid[r, c] == '':  # Found an empty spot
            return r, c

        if dist >= max_dist:  # Avoid searching too far
            continue

        # Check all 8 possible neighboring directions
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols and (nr, nc) not in visited:
                queue.append((nr, nc, dist + 1))
                visited.add((nr, nc))

    return None  # No empty spot found (shouldn't happen in a large enough grid)

def populate_grid_with_tiebreaking(elements, grid_size_factor=4):
    """
    Populates a grid with elements, resolving conflicts by moving duplicates
    to the nearest available space.
    """
    # Get original coordinate limits
    max_x = max(e[1] for e in elements) + 1
    max_y = max(e[2] for e in elements) + 1

    # Create an expanded grid to allow spreading
    grid = np.full((grid_size_factor * max_x, grid_size_factor * max_y), '', dtype=object)

    for label, x, y in elements:
        # Scale up the coordinates for better spread
        x *= grid_size_factor // 2
        y *= grid_size_factor // 2

        if grid[x, y] == '':  # Place if empty
            grid[x, y] = label
        else:  # Resolve conflict by finding a nearby spot
            
            new_x, new_y = find_nearest_empty(grid, x, y)
            if new_x is not None:
                grid[new_x, new_y] = label
            else:
                print(f"Warning: No space found for {label}")

    return grid

# Example: List of elements with some duplicate coordinates
elements = coords_dict

# Generate grid with tie-breaking
result_grid = populate_grid_with_tiebreaking(elements)
for row in result_grid:
    print(row)

def shift_rows_left(grid):
    """
    Shifts all non-empty elements in each row to the left, ensuring
    that every row starts with a channel identifier ('ChX') rather than ''.
    """
    new_grid = np.full_like(grid, '', dtype=object)  # Create an empty grid of the same shape

    for i, row in enumerate(grid):
        non_empty_elements = [elem for elem in row if elem]  # Filter out empty strings
        new_grid[i, :len(non_empty_elements)] = non_empty_elements  # Place elements at the start
    
    return new_grid

# Shift rows to the left
result_grid = shift_rows_left(result_grid)

# Display the final grid

# remove empy row and columns
result_grid = result_grid[~np.all(result_grid == '', axis=1)]
result_grid = result_grid[:, ~np.all(result_grid == '', axis=0)]


['' '' np.int64(5) np.int64(3) np.int64(1) np.int64(4) '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
['' '' np.int64(6) np.int64(7) np.int64(2) np.int64(8) np.int64(20) ''
 np.int64(12) '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
['' np.int64(29) np.int64(25) np.int64(30) np.int64(21) '' np.int64(19)
 np.int64(13) np.int64(11) np.int64(14) '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '']
['' '' np.int64(26) '' np.int64(22) '' np.int64(18) '' np.int64(10) '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
['' np.int64(31) np.int64(27) np.int64(32) np.int64(23) '' np.int64(17)
 np.int64(16) np.int64(9) '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '']
[np.int64(56) '' np.int64(28) '' np.int64(24) '' '' '' np.int64(15) ''
 np.int64(122) '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
[np.int64(55) '' '' '' '' '' '' '' np.int64(99) '' np.int64(121) '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '']
[np.int64(58) '' '' '' '' 

In [93]:
import numpy as np

def compact_grid(grid):
    """
    Removes all empty elements from the grid while preserving relative spacing as much as possible.
    """
    # Remove empty elements from each row
    cleaned_rows = [list(filter(lambda x: x != '', row)) for row in grid]

    # Determine the maximum row length in the new compacted form
    max_cols = max(len(row) for row in cleaned_rows)

    # Create a new NumPy array with a compact shape
    compacted_grid = np.full((len(cleaned_rows), max_cols), '', dtype=object)

    # Fill the new compacted grid while keeping relative order
    for i, row in enumerate(cleaned_rows):
        compacted_grid[i, :len(row)] = row

    return compacted_grid

# Apply function to the cleaned grid
new_compact_grid = compact_grid(result_grid)

# Print output
for row in new_compact_grid:
    print(row)


[np.int64(5) np.int64(3) np.int64(1) np.int64(4) '' '' '' '' '']
[np.int64(6) np.int64(7) np.int64(2) np.int64(8) np.int64(20) np.int64(12)
 '' '' '']
[np.int64(29) np.int64(25) np.int64(30) np.int64(21) np.int64(19)
 np.int64(13) np.int64(11) np.int64(14) '']
[np.int64(26) np.int64(22) np.int64(18) np.int64(10) '' '' '' '' '']
[np.int64(31) np.int64(27) np.int64(32) np.int64(23) np.int64(17)
 np.int64(16) np.int64(9) '' '']
[np.int64(56) np.int64(28) np.int64(24) np.int64(15) np.int64(122) '' ''
 '' '']
[np.int64(55) np.int64(99) np.int64(121) '' '' '' '' '' '']
[np.int64(58) np.int64(100) np.int64(124) np.int64(106) np.int64(126) ''
 '' '' '']
[np.int64(57) np.int64(60) np.int64(103) np.int64(101) np.int64(104)
 np.int64(105) np.int64(123) np.int64(125) '']
[np.int64(59) np.int64(65) np.int64(42) np.int64(164) np.int64(102)
 np.int64(165) np.int64(110) np.int64(132) '']
[np.int64(61) np.int64(44) np.int64(41) np.int64(163) np.int64(107)
 np.int64(127) np.int64(109) np.int64(128) np.i

In [94]:
import numpy as np

def remove_empty_and_compact(grid):
    """
    Removes all empty elements from the grid and restructures it into a compact 2D array,
    maintaining relative spatial relationships as much as possible.
    """
    # Flatten the grid and filter out empty elements
    non_empty_elements = [elem for row in grid for elem in row if elem]

    # Estimate a new grid shape (rows, cols) close to the original aspect ratio
    original_shape = grid.shape
    new_rows = int(np.sqrt(len(non_empty_elements)))  # Approximate square-like structure
    new_cols = int(np.ceil(len(non_empty_elements) / new_rows))  # Balance width

    # Create a new compact grid
    compact_grid = np.full((new_rows, new_cols), '', dtype=object)

    # Fill the compact grid with elements while maintaining order
    index = 0
    for r in range(new_rows):
        for c in range(new_cols):
            if index < len(non_empty_elements):
                compact_grid[r, c] = non_empty_elements[index]
                index += 1

    return compact_grid

# Apply function to the grid
fully_compact_grid = remove_empty_and_compact(result_grid)

# Print output
for row in fully_compact_grid:
    print(row)
pd.DataFrame(fully_compact_grid).to_csv('data/fully_compact_grid.csv', index=False)

[np.int64(5) np.int64(3) np.int64(1) np.int64(4) np.int64(6) np.int64(7)
 np.int64(2) np.int64(8) np.int64(20) np.int64(12) np.int64(29)
 np.int64(25) np.int64(30)]
[np.int64(21) np.int64(19) np.int64(13) np.int64(11) np.int64(14)
 np.int64(26) np.int64(22) np.int64(18) np.int64(10) np.int64(31)
 np.int64(27) np.int64(32) np.int64(23)]
[np.int64(17) np.int64(16) np.int64(9) np.int64(56) np.int64(28)
 np.int64(24) np.int64(15) np.int64(122) np.int64(55) np.int64(99)
 np.int64(121) np.int64(58) np.int64(100)]
[np.int64(124) np.int64(106) np.int64(126) np.int64(57) np.int64(60)
 np.int64(103) np.int64(101) np.int64(104) np.int64(105) np.int64(123)
 np.int64(125) np.int64(59) np.int64(65)]
[np.int64(42) np.int64(164) np.int64(102) np.int64(165) np.int64(110)
 np.int64(132) np.int64(61) np.int64(44) np.int64(41) np.int64(163)
 np.int64(107) np.int64(127) np.int64(109)]
[np.int64(128) np.int64(131) np.int64(62) np.int64(66) np.int64(43)
 np.int64(152) np.int64(166) np.int64(108) np.int64(112